# RAG = Vector Embedding + LLM

[RAG](https://python.langchain.com/v0.1/docs/modules/data_connection/)는 Retrieval-Augmented Generation의 약자로 LLM 모델이 특정 문서에 근거하여 보다 **정확한 답변**을 할 수 있도록 하는 기법. 자체적으로 구축한 DB나 외부 DB에서 사용자 질의에 대해 답변할 수 있는 근거 문서를 찾고, 이를 질문과 함께 LLM에게 전달하여 답변하게 한다. 이렇게 하므로써 LLM 모델이 학습하지 않은 내용에 대해 응답할 수있으며, 환각 현상 (hallucination) 이 줄어든 답변을 생성할 수있다.

## RAG와 Fine tuning

- **Fine tuning**
    - 범용적인 대용량의 데이터로 학습된 사전 학습 모델(pre-trained model)에 모델이 처리하기 바라는 특정 도메인 데이터를 추로 학습시켜 맞춤형 모델로 업데이트 하는 방식이다.
    - **장점**
      - 도메인 최적화되어 높은 성능을 낼 수있다.
    - **단점**
      - 학습에 시간과 노력이 크게 든다. 
      - 실시간 정보가 반영되지 못한다. 그래서 새로운 데이터가 업데이트 되는 경우 재학습을 시켜야 한다.
- **RAG**
  - 장점
    - 최신정보를 반영하는 것이 쉽다. 

## RAG 작동 단계
1. 정보 저장
    1. **Load**: 원본 text 데이터를 읽어온다.
    2. **Split**: Load한 데이터를 작은 덩어리(chunk)로 나눈다.
    3. **Embedding**: text를 embedding vector로 변환한다.
    4. **Store**: Embedding된 chunk를 vector db에 저장한다.
   
![rag](figures/rag1.png)

2. 탐색, 질의, 생성
    1. **Retrieve**: 질문과 관련있는 vector db에서 검색한다.
    2. **Query**: 질문과 검색한 내용을 prompt로 만들어 LLM에 질의한다.
    3. **Generation**: LLM은 받은 query에 대한 답변을 생성한다.
   
- RAG 흐름
  
![Retrieve and Generation](figures/rag2.png)


# Loader

![rag_load](figures/rag_load.png)

- 다양한 문서 형식(format)에 맞춰 읽어오는 다양한 Loader 들이 있다.
  - https://python.langchain.com/docs/how_to/#document-loaders
  - 3rd party library(ppt, github 등등 다양한 3rd party lib도 있음. ) 들과 연동해 다양한 형식의 문서를 읽어올 수 있다.
    - https://python.langchain.com/docs/integrations/document_loaders/
- **모든 Loader는 기본적으로 동일한 interface(사용법)로 호출할 수있다.**
- **반환타입**
  - **list[Document]**
  - Load 한 문서는 Document객체에 정보들을 넣는다. 여러 문서를 읽을 수 있기 대문에 list에 묶어서 반환한다.
    - **Document 속성**
      - page_content: 문서의 내용
      - metadata(option): 문서에 대한 메타데이터(정보)를 dict 형태로 저장한다. 
      - id(option): 문서의 고유 id
## 주요 Loader

### Text file
- TextLoader 이용

In [1]:
from  langchain_community.document_loaders import TextLoader
path = r'data/olympic.txt'

# 1. Loader 객체 생성
loader = TextLoader(path, encoding="utf-8")
# 2. Loader를 이용해서 파일 내용을 읽어오기(load)
docs = loader.load()


In [2]:
print(type(docs), len(docs))
print(type(docs[0]))
docs[0]
print(docs[0].page_content) # 읽어온 내용.

<class 'list'> 1
<class 'langchain_core.documents.base.Document'>
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.
또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플

In [3]:
from pprint import pprint
pprint(docs[0].metadata)

{'source': 'data/olympic.txt'}


In [ ]:
from langchain_core.documents import Document
full_text = ""
for doc in docs:
    full_text += doc.page_content

metadata =("source": docs[0].metadata.get{"source"})
d = Document(pate_content=full_text, metadata=metadata)

SyntaxError: invalid syntax (1578609233.py, line 6)

### PDF
- PyPDF, Pymupdf 등 다양한 3rd party library를 이용해 pdf 문서를 읽어온다.
  - https://python.langchain.com/docs/integrations/document_loaders/#pdfs
- 설치 패키지
  - `pip install pypdf -qU`
  - `pip install pymupdf -qU`

In [4]:
from langchain_community.document_loaders import PyPDFLoader
path = "data/novel/동백꽃_김유정.pdf"

loader = PyPDFLoader(file_path=path)
docs = loader.load()
print(type(docs))
print(len(docs))  #   페이지 단위로 나눠서 Document를 생성.
docs[0]
print(docs[2].page_content)
docs[1].metadata

<class 'list'>
16
3 
쌩이질을  하는  것은  다  뭐냐 . 그것도  발소리를  죽여  가지고
등뒤로  살며시  와서 ,
" 얘 ! 너  혼자만  일하니 ?"
하고  긴치  않는  수작을  하는  것이다 .
어 제까지도  저와  나는  이야 기도  잘  않고  서로  만나도  본체
만  척하고  이렇게  점잖게  지내던  터이련만  오 늘로  갑작스레
대견해졌음은  웬일인가 . 항차  망아 지만  한  계집애 가  남  일하
는  놈  보구 …….
" 그럼  혼자  하지  떼루  하디 ?"
내가  이렇게  내배앝 는  소리를  하니까 ,
" 너  일하기  좋니 ?"
또는 ,
" 한여 름이나  되거든  하지  벌써  울타리를  하니 ?"
잔소리를  두루  늘어 놓다가  남이  들을까  봐  손으로  입을  틀어
막고는  그  속에 서  깔깔댄다 . 별로  우스울  것도  없 는데  날씨
가  풀리더니  이  놈의  계집애 가  미쳤나  하고  의심하였 다 . 게
다가  조금  뒤에 는  제  집께를  할금  할금  돌아 보더니  행주치마
의  속으로  꼈던  바른손을  뽑아 서  나의  턱밑으로  불쑥  내미는
것이다 . 언 제  구웠는  지  더운  김이  홱  끼치는  굵은  감자  세
개가  손에  뿌듯이  쥐였 다 .


{'source': 'data/novel/동백꽃_김유정.pdf', 'page': 1}

In [5]:
### Pymupdf
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(path)
docs = loader.load()
print(type(docs))
print(len(docs))  # page 별로 나눠서 읽는다.
docs[0]
print(docs[2].page_content)
docs[1].metadata


<class 'list'>
16
3
쌩이질을 하는 것은 다 뭐냐. 그것도 발소리를 죽여 가지고
등뒤로 살며시 와서,
"얘! 너 혼자만 일하니?"
하고 긴치 않는 수작을 하는 것이다.
어제까지도 저와 나는 이야기도 잘 않고 서로 만나도 본체
만 척하고 이렇게 점잖게 지내던 터이련만 오늘로 갑작스레
대견해졌음은 웬일인가. 항차 망아지만 한 계집애가 남 일하
는 놈 보구…….
"그럼 혼자 하지 떼루 하디?"
내가 이렇게 내배앝는 소리를 하니까,
"너 일하기 좋니?"
또는,
"한여름이나 되거든 하지 벌써 울타리를 하니?"
잔소리를 두루 늘어놓다가 남이 들을까 봐 손으로 입을 틀어
막고는 그 속에서 깔깔댄다. 별로 우스울 것도 없는데 날씨
가 풀리더니 이 놈의 계집애가 미쳤나 하고 의심하였다. 게
다가 조금 뒤에는 제 집께를 할금 할금 돌아보더니 행주치마
의 속으로 꼈던 바른손을 뽑아서 나의 턱밑으로 불쑥 내미는
것이다. 언제 구웠는 지 더운 김이 홱 끼치는 굵은 감자 세
개가 손에 뿌듯이 쥐였다.



{'source': 'data/novel/동백꽃_김유정.pdf',
 'file_path': 'data/novel/동백꽃_김유정.pdf',
 'page': 1,
 'total_pages': 16,
 'format': 'PDF 1.4',
 'title': '동백꽃',
 'author': 'Unknown',
 'subject': '',
 'keywords': '',
 'creator': 'Wikisource',
 'producer': 'Wikisource',
 'creationDate': "D:20241124070355+00'00'",
 'modDate': "D:20241124070356+00'00'",
 'trapped': ''}





위키백과, 우리 모두의 백과사전.




FIFA 월드컵종목축구개최국FIFA 가맹국들주최FIFA선수단211개국 (2022년 지역 예선)48개국 (본선)역사설립1930년초대 우승 우루과이 (1930년)최다 우승 브라질 (5회)최근 우승 아르헨티나 (2022년) 2026년 FIFA 월드컵
2022년 FIFA 월드컵에서 우승한 아르헨티나.
FIFA 월드컵(영어: FIFA World Cup)은 축구 국제 기구인 국제 축구 연맹(FIFA)에 가맹한 축구 협회(연맹)의 남자 축구 국가대표팀만 참가하는 국제 축구 대회이다. 일반적으로 월드컵 축구나 월드컵이라고도 한다.
4년마다 열리는 월드컵은 1930년에 첫 대회가 열렸다. 1942년과 1946년 대회는 제2차 세계 대전으로 인하여 열리지 못했다. 대회는 예선 무대와 본선 무대 등 두 부분으로 나뉜다. 예선 무대는 본선에 진출할 32팀을 가려내기 위해 본선 보다 3년 일찍 시작한다. 현재 본선은 개최국 경기장에서 한 달 남짓 32개 팀이 우승을 놓고 경쟁하는 방식으로 진행된다. 월드컵 결승전은 세계에서 가장 많은 사람이 시청하는 단일 스포츠 경기이다. 어림잡아 7억 1,510만 명이 2006년 FIFA 월드컵 결승전을 시청했다고 한다.[1]
총 20번 대회가 열리는 동안 8팀이 우승을 차지했다. 가장 우승 횟수가 많은 팀은 브라질로 총 다섯 번의 우승컵을 들어올렸다. 그 다음으로 이탈리아와 독일이 네 번[2], 아르헨티나가 세 번[3], 그리고 초대 우승팀인 우루과이와 프랑스가 각각 두 차례씩, 잉글랜드와 스페인가 각각 한 차례씩 우승을 차지했다. 네덜란드는 준우승만 3번(1974·1978·2010)했다.대한민국은 4위가 최고 기록이다.(2002)
가장 최근에 열린 2022년 FIFA 월드컵은 카타르에서 열렸고, 아르헨티나가 우승했다. 2026년은 캐나다/멕시코/미국에서 개최된다.
세계적인 종합 스포츠 행사 중 하나인 올림픽과 달리 월드컵은 단일 종목 대회다. 그리고 올림픽은 고대 그리스의 전통을 따라 한 도시를 

### CSV
- CSVLoader

In [6]:
# csv 형식의 text 파일 loading.
# 행별로 document 가 생성됨.
from langchain_community.document_loaders import CSVLoader
path = 'data/boston_hosing.csv'
loader = CSVLoader(path)
docs = loader.load()
print(type(docs))
print(len(docs))  # page 별로 나눠서 읽는다.
docs[0]
print(docs[0].page_content)

<class 'list'>
506
CRIM: 0.00632
ZN: 18.0
INDUS: 2.31
CHAS: 0.0
NOX: 0.538
RM: 6.575
AGE: 65.2
DIS: 4.09
RAD: 1.0
TAX: 296.0
PTRATIO: 15.3
B: 396.9
LSTAT: 4.98
MEDV: 24.0


### Web

- WebBaseLoader 이용
- https://python.langchain.com/docs/how_to/document_loader_web/

In [7]:
import os
os.environ['user_agent']="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"

In [8]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://ko.wikipedia.org/wiki/FIFA_%EC%9B%94%EB%93%9C%EC%BB%B5",  # FIFA_월드컵
    "https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC",  # 축구
    "https://ko.wikipedia.org/wiki/%EA%B5%AD%EC%A0%9C_%EC%B6%95%EA%B5%AC_%EC%97%B0%EB%A7%B9"  # 국제 축구연맹
]
loader = WebBaseLoader(
    web_path=urls,   # urls 별로 문서를 생성.  그 문서의 text를 추출한다.
    header_template={
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
)
docs = loader.load()
print(type(docs), len(docs))
print(docs[0].metadata)
print(docs[0].page_content)

<class 'list'> 3
{'source': 'https://ko.wikipedia.org/wiki/FIFA_%EC%9B%94%EB%93%9C%EC%BB%B5', 'title': 'FIFA 월드컵 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}




FIFA 월드컵 - 위키백과, 우리 모두의 백과사전




































본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색






















보이기
















기부

계정 만들기

로그인








개인 도구





기부 계정 만들기 로그인





		로그아웃한 편집자를 위한 문서 더 알아보기



기여토론




























목차
사이드바로 이동
숨기기




처음 위치





1
역사




역사 하위섹션 토글하기





1.1
이전의 국제 대회들








1.2
첫 번째 월드컵








1.3
월드컵 대회가 맞은 시련








1.4
시련 이후 계속되는 성장








1.5
다른 FIFA 대회들










2
트로피








3
예선








4
대륙별 본선 진출국 배분




대륙별 본선 진출국 배분 하위섹션 토글하기





4.1
본선










5
개최국 선정








6
조 편성




조 편성 하위섹션 토글하기





6.1
톱시드 편성










7
미디어 중계








8
역대 대회 결과




역대 대회 결과 하위섹션 토글하기





8.1
역대 결승전 & 3·4위 결정전








8.2
국가별 우승 횟수


In [9]:
# 특정 위치의 내용들만 조회 - SoupStainer
# Soupstainer("a")  # a 태그들의 내용만 조회 .(태그명으로 제한)
# SoupStrainer("a", href=True)  # a 태그중 href 속성이 있는 태그들의 내용만 조회.
# SoupStriner(attrs={"class":"content", 속성명:속성값})  # 모든 태그들 중 class 속성 = content 속성을 가진 태그들만 조회.

In [20]:
import bs4
loader = WebBaseLoader(
    web_path=urls,
    bs_kwargs={  # beautifulsoup 설정을 key: value로 설정.
        "parse_only": bs4.SoupStrainer(attrs={"class":"vector-body"}),   # 조건(clss='vector-body') 태그안의 내용들만 가져온다.
    }
)
docs = loader.load()
print(docs[0].page_content)





위키백과, 우리 모두의 백과사전.




FIFA 월드컵종목축구개최국FIFA 가맹국들주최FIFA선수단211개국 (2022년 지역 예선)48개국 (본선)역사설립1930년초대 우승 우루과이 (1930년)최다 우승 브라질 (5회)최근 우승 아르헨티나 (2022년) 2026년 FIFA 월드컵
2022년 FIFA 월드컵에서 우승한 아르헨티나.
FIFA 월드컵(영어: FIFA World Cup)은 축구 국제 기구인 국제 축구 연맹(FIFA)에 가맹한 축구 협회(연맹)의 남자 축구 국가대표팀만 참가하는 국제 축구 대회이다. 일반적으로 월드컵 축구나 월드컵이라고도 한다.
4년마다 열리는 월드컵은 1930년에 첫 대회가 열렸다. 1942년과 1946년 대회는 제2차 세계 대전으로 인하여 열리지 못했다. 대회는 예선 무대와 본선 무대 등 두 부분으로 나뉜다. 예선 무대는 본선에 진출할 32팀을 가려내기 위해 본선 보다 3년 일찍 시작한다. 현재 본선은 개최국 경기장에서 한 달 남짓 32개 팀이 우승을 놓고 경쟁하는 방식으로 진행된다. 월드컵 결승전은 세계에서 가장 많은 사람이 시청하는 단일 스포츠 경기이다. 어림잡아 7억 1,510만 명이 2006년 FIFA 월드컵 결승전을 시청했다고 한다.[1]
총 20번 대회가 열리는 동안 8팀이 우승을 차지했다. 가장 우승 횟수가 많은 팀은 브라질로 총 다섯 번의 우승컵을 들어올렸다. 그 다음으로 이탈리아와 독일이 네 번[2], 아르헨티나가 세 번[3], 그리고 초대 우승팀인 우루과이와 프랑스가 각각 두 차례씩, 잉글랜드와 스페인가 각각 한 차례씩 우승을 차지했다. 네덜란드는 준우승만 3번(1974·1978·2010)했다.대한민국은 4위가 최고 기록이다.(2002)
가장 최근에 열린 2022년 FIFA 월드컵은 카타르에서 열렸고, 아르헨티나가 우승했다. 2026년은 캐나다/멕시코/미국에서 개최된다.
세계적인 종합 스포츠 행사 중 하나인 올림픽과 달리 월드컵은 단일 종목 대회다. 그리고 올림픽은 고대 그리스의 전통을 따라 한 도시를 

In [21]:
urls = [
    "https://ko.wikipedia.org/wiki/FIFA_%EC%9B%94%EB%93%9C%EC%BB%B5",  # FIFA_월드컵
    "https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC",  # 축구
    "https://ko.wikipedia.org/wiki/%EA%B5%AD%EC%A0%9C_%EC%B6%95%EA%B5%AC_%EC%97%B0%EB%A7%B9"  # 국제 축구연맹
    "https://n.news.naver.com/article/057/0001859292?cds=news_media_pc&type=editn"  # 계엄 또 못할리 없음.
]

In [22]:
def wiki_and_naver(tag_name:str, tag_attribute:dict)->bool:
    # 모든 tag들을 조회할때 마다, 이 함수에 태그이름과  그 태그의 속성들을 전달.
    # True가 return 되는 태그의 content만 조회대상.
    # {"속성명":"속성값"}
    return tag_attribute.get("id") == "dic_area" or (tag_attribute.get("class") is not None and 'vector-body' in tag_attribute.get('class'))
    # Naver 뉴스기사는 id="dic_area" 인 태그 내에 있다.

loader = WebBaseLoader(
    web_path=urls,
    bs_kwargs={  # beautifulsoup 설정을 key: value로 설정.
        "parse_only": bs4.SoupStrainer(wiki_and_naver),   # 조건(clss='vector-body') 태그안의 내용들만 가져온다.
    }
)
docs = loader.load()
docs[3]

IndexError: list index out of range

In [13]:
docs[3]

IndexError: list index out of range

### UnstructuredLoader
- 다양한 비정형 문서들을 읽어 오는 Unstructured 를 사용해, 다양한 형식의 문서들을 load 해 RAG, 모델 파인튜닝에 적용할 수있게 한다.
  - 지원 파일 형식: "csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"

- **다양한 형식의 파일로 부터 text를 로딩**해야 할 경우 유용하다. 
- Local에 library를 설치해서 사용하거나,  Unstructured 가 제공하는 API service를 사용할 수 있다.
  - https://docs.unstructured.io
- 텍스트 파일, PDF, 이미지, HTML, XML, ms-office(word, ppt), epub 등 다양한 비정형 데이터 파일을 처리할 수 있다.
  - 설치, 지원 문서: https://docs.unstructured.io/open-source/installation/full-installation
  - Langchain 문서: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file

> - UnstructuredLoader PDF Load 시 Document 분할 기준
>     -  문서의 구조와 콘텐츠를 기반으로 텍스트를 분할해 Document에 넣는다.
>     -  분할 기준
>        - 헤더(Header): 문서의 제목이나 섹션 제목 등
>        - 본문 텍스트(NarrativeText): 일반적인 문단이나 설명문
>        - 표(Table): 데이터가 표 형식으로 구성된 부분
>        - 리스트(List): 순서가 있거나 없는 목록
>        - 이미지(Image): 사진이나 그래픽 요소

#### 설치할 프로그램
- poppler
  - pdf 파일을 text로 변환하기 위해 필요한 프로그램
  - windows: https://github.com/oschwartz10612/poppler-windows/releases/ 에서 최신 버전 다운로드 후 압축 풀어서 설치.
    - 환경변수 Path에 "설치경로\Library\bin" 을 추가. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install poppler`
  - Linux: `sudo apt-get install poppler-utils`
    - 고급시스템설정 / 환경변수 / path / 새로만들기 => "C:\Users\Playdata\Documents\class\homework\poppler-24.08.0\Library\bin" 삽입 


- tesseract-ocr
  - OCR 라이브러리로 pdf 이미지를 text로 변환하기 위해 필요한 프로그램 
  - windows: https://github.com/UB-Mannheim/tesseract/wiki 에서 다운받아 설치. 
    - 환경변수 Path에 설치 경로("C:\Program Files\Tesseract-OCR") 추가 한다. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install tesseract`
  - linux(unbuntu): `sudo apt install tesseract-ocr`  
    - 고급시스템설정 / 환경변수 / path / 새로만들기 => "C:\Program Files\Tesseract-OCR"
    
    
- 설치 패키지
  - `pip install "unstructured
  - `pip install "unstructured[pdf]" -qU`
  - libmagic 설치
      - windows: `pip install python-magic-bin -qU`
      - macOS: `brew install libmagic`
      - linux(ubuntu): `sudo apt-get install libmagic-dev`
  - `pip install langchain-unstructured -qU`

> pip install --force-reinstall  python-magic-bin  

> pip install onnx  

> pip install onnxruntime==1.17.0  

> pip install unstructured[pdf]  

> pip install langchain-unstructured

In [1]:
import os
os.environ["path"] += r";C:\Users\Playdata\Documents\class\homework\poppler-24.08.0\Library\bin"

# C:\Users\Playdata\Documents\class\homework\poppler-24.08.0\Library\bin

In [15]:
os.environ["path"] += r";C:\Program Files\Tesseract-OCR"

In [5]:
from langchain_unstructured import UnstructuredLoader
# path = "data/"olympic.txt"
path = "data/novel/동백꽃_김유정.pdf"
path = ["data/olympic.txt", "data/novel/금_따는_콩밭_김유정.pdf"]

loader = UnstructuredLoader(path, mode="elements")  # 한개의 file load시 -> 문자열로 파일 경로를 지정.
# 여러개 파일 load -> 경로를 리스트에 묶어서 전달.
docs = loader.load()

In [6]:
len(docs)
docs[1]

Document(metadata={'source': 'data/olympic.txt', 'last_modified': '2024-12-02T09:03:13', 'languages': ['kor'], 'file_directory': 'data', 'filename': 'olympic.txt', 'filetype': 'text/plain', 'category': 'NarrativeText', 'element_id': 'bae765f6ee1f43ccd90ef0f9afd719c9'}, page_content='고대올림픽 고대의 올림픽 경기(올림피아 경기)는 고대 그리스의 여러 도시 국가의 대표선수들이 모여 벌인 일련의 시합이었으며, 육상 경기가 주 종목이지만 격투기와 전차 경기도 열렸다. 그리고 패배하면 죽기도 하였다. 고대 올림픽의 유래는 수수께끼로 남아있다. 잘 알려진 신화로는 헤라클레스와 그의 아버지인 제우스가 올림픽의 창시자였다는 것이다. 전설에 따르면 이 경기를 최초로 \'올림픽\'이라고 부르고, 4년마다 대회를 개최하는 관례를 만든 사람이 헤라클레스라고 한다. 어떤 전설에서는 헤라클레스가 이른바 헤라클레스의 12업을 달성한 뒤에 제우스를 기리고자 올림픽 경기장을 지었다고 한다. 경기장이 완성되자 헤라클레스는 일직선으로 200 걸음을 걸었으며, 이 거리를 "스타디온"이라 불렀는데, 후에 이것이 길이 단위인 \'스타디온\'(그리스어: στάδιον → 라틴어: 영어: stadium)이 되었다. 또 다른 설로는 \'올림픽 휴전\'(그리스어: ἐκεχειρία 에케케이리아[*])이라는 고대 그리스의 관념이 최초의 올림피아 경기와 관련이 있다고 한다. \'올림픽 휴전\'이란 어느 도시 국가라도 올림피아 경기 기간 중에 다른 나라를 침범하면 그에 대한 응징을 받을 수 있다는 뜻으로, "올림픽 기간에는 전쟁하지 말 것"으로 요약할 수 있다. 고대 올림피아 경기가 처음 열린 시점은 보통 기원전 776년으로 인정되고 있는데, 이 연대는 그리스 올림피아에서 발견된 비문에 근

In [7]:
print(docs[5].page_content)

패럴림픽 패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 스포츠 대회로, 장애인 올림픽으로 불린다. 1948년에 루드비히 구트만 경(Sir Ludwig Guttman)은 제2차 세계대전에 참전한 군인들의 사회 복귀를 위한 일환으로 1948년 런던 올림픽과 동시에 몇몇 병원들을 연합해서 여러 경기를 펼쳤다. 구트만의 세계 휠체어, 신체부자유자대회(World Wheelchair and Amputee Games)로 알려진 이 대회는 매년 열리는 스포츠대회가 되었다. 12년이 넘도록 구트만과 다른 사람들은 스포츠를 상처를 치료하는 방법 중 하나로써 계속 대회 개최에 노력을 기울였다. 로마에서 열린 1960년 하계 올림픽때 구트만은 400명의 선수들을 "Parallel Olympics"에 참가시켰으며 이것이 곧 1회 패럴림픽으로 알려지게 되었다. 그 때부터 패럴림픽은 하계 올림픽이 열린 년도에 열리게 되었다. 서울에서 열린 1988년 하계 올림픽부터는 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 한다.


In [9]:
docs[20].metadata

{'source': 'data/novel/금_따는_콩밭_김유정.pdf',
 'coordinates': {'points': ((121.6874949296875, 299.00498513145845),
   (121.6874949296875, 309.4989845142084),
   (298.30860243247434, 309.4989845142084),
   (298.30860243247434, 299.00498513145845)),
  'system': 'PixelSpace',
  'layout_width': 420.0,
  'layout_height': 594.95996},
 'file_directory': 'data/novel',
 'filename': '금_따는_콩밭_김유정.pdf',
 'languages': ['eng'],
 'last_modified': '2024-12-02T09:03:13',
 'page_number': 1,
 'parent_id': '85f409ed3ecd6c94b038c17f0c22dfc5',
 'filetype': 'application/pdf',
 'category': 'NarrativeText',
 'element_id': '6fbc6aa422ea5c54baec2ea2552fcd8a'}

### Directory 내의 문서파일들 로딩
- DirectoryLoader 이용

In [10]:
from langchain_community.document_loaders import DirectoryLoader
# 1. Directory Loader 객체 생성 -> loading할 문서(들) 위치를 지정.
loader = DirectoryLoader(
    path="./data", # loading한 파일들이 있는 directory. 
    glob=["*.txt", "*.pdf"], # glob 패턴으로 파일형식
    recursive=True, # 하위 디렉토리도 모두 찾는다.  default : False
    # loader_cls=PyMuPDFLoader  # Document Loader를 지정.  (default: UnstructuredLoader)
)
# 2. Load()로 문서 로딩.
docs = loader.load()

In [14]:
len(docs)
print(docs[0])
print(docs[1])

page_content='올림픽 올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다. 또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20